In [29]:
import xlrd 
import pubchempy as pcp
import pickle

import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
import seaborn as sns

from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec, mol2sentence
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg

In [30]:
pickle_in = open("fold-0.pickle","rb")
load_dict =  pickle.load(pickle_in)

data_frame = load_dict['shuffled']

data_frame

,No.,FileHandle,SoluteName,Formula,Subset,Charge,Level1,Level2,Level3,Solvent,...,HP,OP,S,HS,OS,SP,SS,TotalArea,SoluteSMILES,SolventSMILES
1758,1759,0438pho,"diethyl2,4-dichlorophenylthiophosphate",H13C10O3P1S1CL2,[g],0,11,4,0,octanol,...,0.000,16.033,34.582,0.0,0.0,24.737,0.0,331.933,C1C2C(C(C1Cl)Cl)C3(C(=C(C2(C3(Cl)Cl)Cl)Cl)Cl)Cl,CCCCCCCCO
788,789,0071proa,propanal,H6C3O1,[g],0,2,4,1,diethylether,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,124.289,CCC=O,CCOCC
1726,1727,0210dic,"1,1-dichloro-2,2-difluoroethylmethylether",H4C3O1F2CL2,[g],0,9,1,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,178.993,CC1=CC(=O)CC(C1)(C)C,CCCCCCCCO
1939,1940,0110but,butylamine,H11C4N1,[g],0,3,1,0,secbutanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,164.243,CCCCN,CCC(C)O
1467,1468,0008noc,n-octane,H18C8,[g],0,2,1,1,methylformamide,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,243.846,CCCCCCCC,CNC=O
2110,2111,0021eth,ethene,H4C2,[a],0,2,1,4,water,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,87.417,C=C,O
739,740,0087pro,propanoicacid,H6C3O2,[g],0,2,5,0,dichloroethane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,135.630,CCC(=O)O,CC(Cl)Cl
1196,1197,0172Edi,"E-1,2-dichloroethene",H2C2CL2,[g],0,6,2,0,hexadecane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,126.855,C(=CCl)Cl,CCCCCCCCCCCCCCCC
111,112,0150mhy,m-hydroxybenzaldehyde,H6C7O2,[g],0,2,9,0,benzene,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,179.066,C1=CC(=CC(=C1)O)C=O,C1=CC=CC=C1
1820,1821,test4002,diiodomethane,H2C1I2,[g],0,15,0,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,136.531,C(I)I,CCCCCCCCO


In [31]:
# loc = ("MNSol_alldata.xlsx") 
# wb = xlrd.open_workbook(loc) 
# sheet = wb.sheet_by_index(0) 

# solutes = sheet.col_values(2)
# solvents = sheet.col_values(9)
# energies = sheet.col_values(10)

# solute_smis = []
# solvent_smis = []

# for solute in solutes[100:110]:
#     for compound in pcp.get_compounds(solute, 'name'):
#         solute_smis.append(compound.isomeric_smiles)
        
# for solvent in solvents[100:110]:
#     for compound in pcp.get_compounds(solvent, 'name'):
#         solvent_smis.append(compound.isomeric_smiles)

solute_smis = data_frame['SoluteSMILES'].tolist()
solvent_smis = data_frame['SolventSMILES'].tolist()
energies = data_frame['DeltaGsolv'].tolist()

print(len(solute_smis))
print(len(solvent_smis))
print(len(energies))

2530
2530
2530


In [32]:
solutesmis = [Chem.MolFromSmiles(x) for x in solute_smis]
solventsmis = [Chem.MolFromSmiles(x) for x in solvent_smis]

RDKit WARNING: [03:33:18] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [03:33:18] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [03:33:18] WARNING: not removing hydrogen atom without neighbors


In [33]:
solute_sentences = []
solvent_sentences = []

for solute_smi in solutesmis:
    solute_sentences.append(mol2alt_sentence(solute_smi, 1))
    
for solvent_smi in solventsmis:
    solvent_sentences.append(mol2alt_sentence(solvent_smi, 1))
    
print(solute_sentences[6])
print(solvent_sentences[6])

['2246728737', '3542456614', '2245384272', '2064788354', '2246699815', '4278941385', '864942730', '1510328189', '864662311', '1533864325']
['2246728737', '3537119515', '2245273601', '2406620359', '1016841875', '170898322', '1016841875', '170898322']


In [34]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('model_300dim.pkl')

solute_sequences = []
count = 0
sequences = []
solvent_sequences = []
Gsolv = []
l = len(solute_smis)

for i in range(l):
    flag = 0
    solute_substructures = []
    solvent_substructures = []
    
    for identifier in solute_sentences[i]:
        try:
            solute_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            break
    
    for identifier in solvent_sentences[i]:
        try:
            solvent_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            break
            
    if flag == 1:
        count += 1
        continue
    sequences.append((solute_substructures, solvent_substructures))
    solute_sequences.append(solute_substructures)
    solvent_sequences.append(solvent_substructures)
    Gsolv.append(energies[i])
    
    
print(count)
# solute_mol2vecs = [DfVec(x) for x in sentences2vec(solute_sentences, model, unseen='UNK')]
# solvent_mol2vecs = [DfVec(x) for x in sentences2vec(solvent_sentences, model, unseen='UNK')]

401


In [35]:
print(len(solute_sequences))
print(len(solvent_sequences))
print(len(Gsolv))
print(len(sequences))

# for i in solute_sequences:
#     print(np.asarray(i).shape)
# print('\n')
# for i in solvent_sequences:
#     print(np.asarray(i).shape)

2129
2129
2129
2129


In [36]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import math
import time
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer, LayerNorm

In [45]:
class TransformerModel(nn.Module):

    def __init__(self, ninp, nhead, nhid, nlayers, dropout=0):
        super(TransformerModel, self).__init__()
        
        self.model_type = 'Transformer'
        self.solv_src_mask = None
        self.solu_src_mask = None
        self.ninp = ninp
        
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        
        self.solv_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.solu_encoder = TransformerEncoder(encoder_layers, nlayers)
        
        self.fc1 = nn.Linear(600, 300)
        self.fc2 = nn.Linear(300, 1)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.fc1.bias.data.zero_()
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)

    def forward(self, solv_src, solu_src):
        if self.solv_src_mask is None or self.solv_src_mask.size(0) != len(solv_src):
            self.solv_src_mask = self._generate_square_subsequent_mask(len(solv_src))
            
        if self.solu_src_mask is None or self.solu_src_mask.size(0) != len(solu_src):
            self.solu_src_mask = self._generate_square_subsequent_mask(len(solu_src))
        
        solv_src = self.pos_encoder(solv_src)
        solu_src = self.pos_encoder(solu_src)
        
        solv_output = self.solv_encoder(solv_src, self.solv_src_mask)
        solu_output = self.solu_encoder(solu_src, self.solu_src_mask)
        
#         print(solv_output.shape)
        
        solv_output = torch.squeeze(solv_output, 1)
        solu_output = torch.squeeze(solu_output, 1)
        
#         print(solv_output.shape)
        
        # shared attention layer
        
        H = solv_output
        G = solu_output
        
        Hsize = H.size(0)
        Gsize = G.size(0)
        a_score = torch.zeros(Hsize, Gsize)
        
        for i in range(Hsize):
            for j in range(Gsize):
#                 print(a_score[i][j], torch.matmul(H[i],torch.t(G[j])))
                a_score[i][j] = torch.matmul(H[i],torch.t(G[j]))
#                 print(a_score[i][j])
                
        
        a = F.softmax(a_score,1)
        
        P = torch.matmul(a, G)
        Q = torch.matmul(torch.t(a), H)
        
        # maxpooling layer
        u = torch.max(H, P)
        v = torch.max(G, Q)
        
        inpu = torch.sum(u, dim=0)
        inpv = torch.sum(v, dim=0)
        
        inp = torch.cat((inpu, inpv), 0)
        
        # mlp
        x = F.relu(self.fc1(inp))
        output = self.fc2(x)
        
#         print(output.shape)
#         print(output)
        
        return output

In [38]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [39]:
ninp = 300 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 1 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 3 # the number of heads in the multiheadattention models
dropout = 0 # the dropout value

In [40]:
model = TransformerModel(ninp, nhead, nhid, nlayers, dropout)

# PRINTING MODEL & PARAMETERS 
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0, inplace=False)
  )
  (solv_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=200, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=200, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, inplace=False)
      )
    )
  )
  (solu_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=300, out_features=300, bias=True)
        )

In [41]:
learning_rate = 0.000002
nest_momentum = 0.009

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=nest_momentum, nesterov=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

num_epochs = 100
splits = 10

best_model = model
best_loss = float("inf")
losses = []

# Training and 10 fold cross validation
kf = KFold(n_splits = 10)
kf.get_n_splits(sequences)

for train_index, test_index in kf.split(sequences):
    model = TransformerModel(ninp, nhead, nhid, nlayers, dropout)

    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=nest_momentum, nesterov=True)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
    
    seq_tr_va, seq_test = np.asarray(sequences)[train_index], np.asarray(sequences)[test_index]
    E_tr_va, E_test = np.asarray(Gsolv)[train_index], np.asarray(Gsolv)[test_index]
    seq_train, seq_val, E_train, E_val = train_test_split(seq_tr_va, E_tr_va, test_size=0.11)
    
    print("TRAIN:", len(seq_train), "VALIDATION:", len(seq_val), "TEST:", len(seq_test))
    
    #Training
    for epoch in range(num_epochs):   
        model.train()
        for i in range(len(seq_train)):
            solute, solvent = seq_train[i]
            energy = torch.FloatTensor([E_train[i]])
        
            # Forward pass
            solvent = torch.FloatTensor(solvent)
            solute = torch.FloatTensor(solute)
            
            solvent = torch.unsqueeze(solvent, 1)
            solute = torch.unsqueeze(solute, 1)
            
            output = model(solvent, solute)
#             print(output.shape)
            loss = criterion(output, energy)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            
        #Validation
        model.eval()
        outputs = []

        with torch.no_grad():
            for i in range(len(seq_val)):
                solute, solvent = seq_val[i]
                energy = torch.FloatTensor([E_val[i]])
        
                solvent = torch.FloatTensor(solvent)
                solute = torch.FloatTensor(solute)
            
                solvent = torch.unsqueeze(solvent, 1)
                solute = torch.unsqueeze(solute, 1)
            
                output = model(solvent, solute)
            
                outputs.append(output)
                
            val_loss = criterion(torch.squeeze(torch.FloatTensor(E_val), axis=0), torch.squeeze(torch.FloatTensor([outputs]), axis=0))
            print('Validation MSE: {}'.format(val_loss))
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = model
                
            scheduler.step()
            
    #Testing
    model.eval()
    outputs = []

    with torch.no_grad():
        for i in range(len(seq_test)):
            solute, solvent = seq_test[i]
            energy = torch.FloatTensor([E_test[i]])
        
            solvent = torch.FloatTensor(solvent)
            solute = torch.FloatTensor(solute)
            
            solvent = torch.unsqueeze(solvent, 1)
            solute = torch.unsqueeze(solute, 1)
            
            output = best_model(solvent, solute)
            
            outputs.append(output)
                
        test_loss = criterion(torch.squeeze(torch.FloatTensor(E_test), axis=0), torch.squeeze(torch.FloatTensor([outputs]), axis=0))
        print('Test MSE: {}'.format(test_loss))
        losses.append(test_loss)
    

TRAIN: 1705 VALIDATION: 211 TEST: 213
Epoch [1/100], Loss: 176.3651
Epoch [1/100], Loss: 170.1017
Epoch [1/100], Loss: 29.9253
Epoch [1/100], Loss: 28.8006
Epoch [1/100], Loss: 1.9011
Epoch [1/100], Loss: 13.9968
Epoch [1/100], Loss: 24.4427
Epoch [1/100], Loss: 6.6580
Epoch [1/100], Loss: 2.0306
Epoch [1/100], Loss: 2.7389
Epoch [1/100], Loss: 3.0134
Epoch [1/100], Loss: 24.8956
Epoch [1/100], Loss: 2.0041
Epoch [1/100], Loss: 3.1228
Epoch [1/100], Loss: 10.9993
Epoch [1/100], Loss: 3.8905
Epoch [1/100], Loss: 8.5732
Epoch [1/100], Loss: 44.7202
Epoch [1/100], Loss: 11.6564
Epoch [1/100], Loss: 349.0526
Epoch [1/100], Loss: 45.8903
Epoch [1/100], Loss: 0.0026
Epoch [1/100], Loss: 13.8167
Epoch [1/100], Loss: 0.7953
Epoch [1/100], Loss: 3.0341
Epoch [1/100], Loss: 14.0887
Epoch [1/100], Loss: 12.8004
Epoch [1/100], Loss: 1.5998
Epoch [1/100], Loss: 0.5672
Epoch [1/100], Loss: 16.6297
Epoch [1/100], Loss: 0.2246
Epoch [1/100], Loss: 34.7376
Epoch [1/100], Loss: 25.3394
Epoch [1/100], Lo

Epoch [1/100], Loss: 4.1228
Epoch [1/100], Loss: 9.2844
Epoch [1/100], Loss: 9.2292
Epoch [1/100], Loss: 0.0604
Epoch [1/100], Loss: 12.7779
Epoch [1/100], Loss: 5.6645
Epoch [1/100], Loss: 3.3505
Epoch [1/100], Loss: 0.7198
Epoch [1/100], Loss: 4.0139
Epoch [1/100], Loss: 0.5337
Epoch [1/100], Loss: 12.9148
Epoch [1/100], Loss: 0.5091
Epoch [1/100], Loss: 0.8081
Epoch [1/100], Loss: 9.4028
Epoch [1/100], Loss: 3.4632
Epoch [1/100], Loss: 9.3958
Epoch [1/100], Loss: 0.9652
Epoch [1/100], Loss: 0.1595
Epoch [1/100], Loss: 0.0021
Epoch [1/100], Loss: 3.0924
Epoch [1/100], Loss: 1.5631
Epoch [1/100], Loss: 21.5062
Epoch [1/100], Loss: 5.2373
Epoch [1/100], Loss: 1.4108
Epoch [1/100], Loss: 5.0178
Epoch [1/100], Loss: 4.4160
Epoch [1/100], Loss: 31.7220
Epoch [1/100], Loss: 8.4046
Epoch [1/100], Loss: 7.6390
Epoch [1/100], Loss: 0.7212
Epoch [1/100], Loss: 1.2441
Epoch [1/100], Loss: 3.3803
Epoch [1/100], Loss: 0.3091
Epoch [1/100], Loss: 0.7102
Epoch [1/100], Loss: 29.5993
Epoch [1/100], 

Epoch [1/100], Loss: 4.8558
Epoch [1/100], Loss: 0.2416
Epoch [1/100], Loss: 4.9864
Epoch [1/100], Loss: 7.2395
Epoch [1/100], Loss: 33.6091
Epoch [1/100], Loss: 68.1934
Epoch [1/100], Loss: 0.7892
Epoch [1/100], Loss: 38.2331
Epoch [1/100], Loss: 1.1498
Epoch [1/100], Loss: 5.0186
Epoch [1/100], Loss: 0.0227
Epoch [1/100], Loss: 0.0969
Epoch [1/100], Loss: 0.2485
Epoch [1/100], Loss: 16.1842
Epoch [1/100], Loss: 8.1616
Epoch [1/100], Loss: 222.3197
Epoch [1/100], Loss: 35.8142
Epoch [1/100], Loss: 5.5560
Epoch [1/100], Loss: 42.7950
Epoch [1/100], Loss: 9.1349
Epoch [1/100], Loss: 0.1501
Epoch [1/100], Loss: 1.9180
Epoch [1/100], Loss: 0.5568
Epoch [1/100], Loss: 1.2409
Epoch [1/100], Loss: 0.8320
Epoch [1/100], Loss: 0.2907
Epoch [1/100], Loss: 0.0063
Epoch [1/100], Loss: 5.1274
Epoch [1/100], Loss: 8.0620
Epoch [1/100], Loss: 5.3789
Epoch [1/100], Loss: 0.1788
Epoch [1/100], Loss: 1.8032
Epoch [1/100], Loss: 3.6669
Epoch [1/100], Loss: 5.1939
Epoch [1/100], Loss: 0.3010
Epoch [1/100

Epoch [1/100], Loss: 1.7737
Epoch [1/100], Loss: 0.2446
Epoch [1/100], Loss: 4.5023
Epoch [1/100], Loss: 29.9332
Epoch [1/100], Loss: 0.2902
Epoch [1/100], Loss: 0.1311
Epoch [1/100], Loss: 2.7183
Epoch [1/100], Loss: 11.2146
Epoch [1/100], Loss: 10.2610
Epoch [1/100], Loss: 0.0443
Epoch [1/100], Loss: 4.3616
Epoch [1/100], Loss: 1.7360
Epoch [1/100], Loss: 1.4987
Epoch [1/100], Loss: 1.7377
Epoch [1/100], Loss: 0.4195
Epoch [1/100], Loss: 23.7526
Epoch [1/100], Loss: 0.0300
Epoch [1/100], Loss: 3.6442
Epoch [1/100], Loss: 9.6195
Epoch [1/100], Loss: 20.9620
Epoch [1/100], Loss: 3.9201
Epoch [1/100], Loss: 0.1930
Epoch [1/100], Loss: 2.3406
Epoch [1/100], Loss: 15.2346
Epoch [1/100], Loss: 13.9625
Epoch [1/100], Loss: 1.8187
Epoch [1/100], Loss: 3.7299
Epoch [1/100], Loss: 0.3890
Epoch [1/100], Loss: 35.3258
Epoch [1/100], Loss: 4.4583
Epoch [1/100], Loss: 5.7572
Epoch [1/100], Loss: 0.2070
Epoch [1/100], Loss: 2.9157
Epoch [1/100], Loss: 2.1206
Epoch [1/100], Loss: 7.3585
Epoch [1/100

Epoch [1/100], Loss: 0.1579
Epoch [1/100], Loss: 1.7302
Epoch [1/100], Loss: 136.1389
Epoch [1/100], Loss: 4.1326
Epoch [1/100], Loss: 0.8670
Epoch [1/100], Loss: 0.0624
Epoch [1/100], Loss: 3.6803
Epoch [1/100], Loss: 0.5565
Epoch [1/100], Loss: 2.9087
Epoch [1/100], Loss: 1.6608
Epoch [1/100], Loss: 3.0957
Epoch [1/100], Loss: 0.9283
Epoch [1/100], Loss: 0.0000
Epoch [1/100], Loss: 14.1701
Epoch [1/100], Loss: 3.3818
Epoch [1/100], Loss: 13.1331
Epoch [1/100], Loss: 1.3123
Epoch [1/100], Loss: 18.8746
Epoch [1/100], Loss: 1.9951
Epoch [1/100], Loss: 6.0265
Epoch [1/100], Loss: 5.0287
Epoch [1/100], Loss: 1.2267
Epoch [1/100], Loss: 38.2690
Epoch [1/100], Loss: 11.8179
Epoch [1/100], Loss: 0.6566
Epoch [1/100], Loss: 0.0218
Epoch [1/100], Loss: 1.6474
Epoch [1/100], Loss: 0.6115
Epoch [1/100], Loss: 1.5200
Epoch [1/100], Loss: 2.2164
Epoch [1/100], Loss: 3.1449
Epoch [1/100], Loss: 0.5456
Epoch [1/100], Loss: 16.9387
Epoch [1/100], Loss: 1.0794
Epoch [1/100], Loss: 11.2655
Epoch [1/10

...

Epoch [9/100], Loss: 0.0789
Epoch [9/100], Loss: 0.0441
Epoch [9/100], Loss: 13.3546
Epoch [9/100], Loss: 0.1332
Epoch [9/100], Loss: 17.4669
Epoch [9/100], Loss: 0.1658
Epoch [9/100], Loss: 4.4202
Epoch [9/100], Loss: 0.0477
Epoch [9/100], Loss: 0.0497
Epoch [9/100], Loss: 3.5264
Epoch [9/100], Loss: 0.3319
Epoch [9/100], Loss: 0.1331
Epoch [9/100], Loss: 0.0102
Epoch [9/100], Loss: 3.9430
Epoch [9/100], Loss: 0.0167
Epoch [9/100], Loss: 1.0977
Epoch [9/100], Loss: 1.3236
Epoch [9/100], Loss: 0.0839
Epoch [9/100], Loss: 0.4487
Epoch [9/100], Loss: 2.8572
Epoch [9/100], Loss: 0.1381
Epoch [9/100], Loss: 6.9682
Epoch [9/100], Loss: 0.1181
Epoch [9/100], Loss: 0.2317
Epoch [9/100], Loss: 2.7573
Epoch [9/100], Loss: 0.3629
Epoch [9/100], Loss: 1.1649
Epoch [9/100], Loss: 0.0936
Epoch [9/100], Loss: 0.0142
Epoch [9/100], Loss: 1.9815
Epoch [9/100], Loss: 0.7375
Epoch [9/100], Loss: 3.7858
Epoch [9/100], Loss: 14.3562
Epoch [9/100], Loss: 1.5981
Epoch [9/100], Loss: 2.1092
Epoch [9/100], Lo

Epoch [9/100], Loss: 0.0004
Epoch [9/100], Loss: 2.7017
Epoch [9/100], Loss: 7.0803
Epoch [9/100], Loss: 2.2854
Epoch [9/100], Loss: 4.4049
Epoch [9/100], Loss: 5.0555
Epoch [9/100], Loss: 60.5081
Epoch [9/100], Loss: 1.7657
Epoch [9/100], Loss: 3.1317
Epoch [9/100], Loss: 0.2952
Epoch [9/100], Loss: 1.3626
Epoch [9/100], Loss: 10.8046
Epoch [9/100], Loss: 1.6462
Epoch [9/100], Loss: 18.1484
Epoch [9/100], Loss: 1.4081
Epoch [9/100], Loss: 0.0261
Epoch [9/100], Loss: 0.0155
Epoch [9/100], Loss: 4.9988
Epoch [9/100], Loss: 0.8581
Epoch [9/100], Loss: 3.8291
Epoch [9/100], Loss: 0.0532
Epoch [9/100], Loss: 0.4086
Epoch [9/100], Loss: 1.0343
Epoch [9/100], Loss: 4.0271
Epoch [9/100], Loss: 0.0402
Epoch [9/100], Loss: 1.2926
Epoch [9/100], Loss: 1.5292
Epoch [9/100], Loss: 0.9123
Epoch [9/100], Loss: 0.0050
Epoch [9/100], Loss: 8.4257
Epoch [9/100], Loss: 0.0148
Epoch [9/100], Loss: 0.3235
Epoch [9/100], Loss: 0.9629
Epoch [9/100], Loss: 2.0756
Epoch [9/100], Loss: 0.0151
Epoch [9/100], Lo

Epoch [9/100], Loss: 1.1376
Epoch [9/100], Loss: 14.0694
Epoch [9/100], Loss: 2.0319
Epoch [9/100], Loss: 15.1515
Epoch [9/100], Loss: 5.1015
Epoch [9/100], Loss: 0.4071
Epoch [9/100], Loss: 0.3950
Epoch [9/100], Loss: 0.4119
Epoch [9/100], Loss: 0.1673
Epoch [9/100], Loss: 2.1243
Epoch [9/100], Loss: 18.9022
Epoch [9/100], Loss: 0.0896
Epoch [9/100], Loss: 0.1197
Epoch [9/100], Loss: 10.2221
Epoch [9/100], Loss: 0.4178
Epoch [9/100], Loss: 1.3152
Epoch [9/100], Loss: 0.3176
Epoch [9/100], Loss: 2.8905
Epoch [9/100], Loss: 0.5212
Epoch [9/100], Loss: 0.0063
Epoch [9/100], Loss: 3.1571
Epoch [9/100], Loss: 0.4308
Epoch [9/100], Loss: 0.6969
Epoch [9/100], Loss: 1.8732
Epoch [9/100], Loss: 0.0058
Epoch [9/100], Loss: 5.1151
Epoch [9/100], Loss: 7.3829
Epoch [9/100], Loss: 2.8500
Epoch [9/100], Loss: 0.1530
Epoch [9/100], Loss: 0.1306
Epoch [9/100], Loss: 1.3589
Epoch [9/100], Loss: 0.7299
Epoch [9/100], Loss: 0.0105
Epoch [9/100], Loss: 17.3109
Epoch [9/100], Loss: 0.1023
Epoch [9/100], 

Epoch [9/100], Loss: 2.3284
Epoch [9/100], Loss: 4.2104
Epoch [9/100], Loss: 0.0660
Epoch [9/100], Loss: 32.0041
Epoch [9/100], Loss: 2.7916
Epoch [9/100], Loss: 11.7330
Epoch [9/100], Loss: 2.5112
Epoch [9/100], Loss: 0.3003
Epoch [9/100], Loss: 2.3571
Epoch [9/100], Loss: 0.7285
Epoch [9/100], Loss: 0.5544
Epoch [9/100], Loss: 0.6367
Epoch [9/100], Loss: 0.0317
Epoch [9/100], Loss: 2.8435
Epoch [9/100], Loss: 0.9397
Epoch [9/100], Loss: 0.0019
Epoch [9/100], Loss: 113.5867
Epoch [9/100], Loss: 2.6235
Epoch [9/100], Loss: 3.3760
Epoch [9/100], Loss: 1.2423
Epoch [9/100], Loss: 0.0358
Epoch [9/100], Loss: 0.1124
Epoch [9/100], Loss: 0.4001
Epoch [9/100], Loss: 0.1191
Epoch [9/100], Loss: 0.6623
Epoch [9/100], Loss: 1.5719
Epoch [9/100], Loss: 0.0145
Epoch [9/100], Loss: 0.0261
Epoch [9/100], Loss: 0.2527
Epoch [9/100], Loss: 0.0539
Epoch [9/100], Loss: 0.1232
Epoch [9/100], Loss: 9.7913
Epoch [9/100], Loss: 3.1731
Epoch [9/100], Loss: 0.0559
Epoch [9/100], Loss: 1.8158
Epoch [9/100], L

Epoch [9/100], Loss: 0.4266
Epoch [9/100], Loss: 0.6768
Epoch [9/100], Loss: 4.9020
Epoch [9/100], Loss: 1.6905
Epoch [9/100], Loss: 0.0056
Epoch [9/100], Loss: 1.6642
Epoch [9/100], Loss: 0.2939
Epoch [9/100], Loss: 2.5396
Epoch [9/100], Loss: 1.9509
Epoch [9/100], Loss: 0.2501
Epoch [9/100], Loss: 0.1869
Epoch [9/100], Loss: 6.8968
Epoch [9/100], Loss: 1.3339
Epoch [9/100], Loss: 2.0559
Epoch [9/100], Loss: 1.9180
Epoch [9/100], Loss: 1.4400
Epoch [9/100], Loss: 0.3998
Epoch [9/100], Loss: 0.0013
Epoch [9/100], Loss: 0.7342
Epoch [9/100], Loss: 0.4157
Epoch [9/100], Loss: 13.1731
Epoch [9/100], Loss: 4.8873
Epoch [9/100], Loss: 1.4350
Epoch [9/100], Loss: 10.3073
Epoch [9/100], Loss: 0.0077
Epoch [9/100], Loss: 0.6650
Epoch [9/100], Loss: 0.0146
Epoch [9/100], Loss: 4.6999
Epoch [9/100], Loss: 28.2081
Epoch [9/100], Loss: 0.0031
Epoch [9/100], Loss: 0.0065
Epoch [9/100], Loss: 0.2759
Epoch [9/100], Loss: 0.4741
Epoch [9/100], Loss: 0.0132
Epoch [9/100], Loss: 10.1599
Epoch [9/100], L

In [ ]:
print(losses)

with open('losses.txt', 'a') as f:
    for item in losses:
        f.write("%s\n" % item)

In [ ]:
# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')